In [1]:
import pandas as pd,numpy as np
from sklearn.metrics import auc

In [2]:
pd.set_option('max_columns',30) 

In [3]:
df_op=pd.read_csv('F:\study\\ml\\DataSet\\TianChengJinRong\\operation_TRAIN.csv')
df_tr=pd.read_csv('F:\study\\ml\\DataSet\\TianChengJinRong\\transaction_TRAIN.csv')
df_tag=pd.read_csv('F:\study\\ml\\DataSet\\TianChengJinRong\\tag_TRAIN.csv')

In [6]:
df_tr.channel.nunique()

4

In [4]:
df_op=df_op.drop_duplicates()
df_tr=df_tr.drop_duplicates()
df_tag=df_tag.drop_duplicates()

In [ ]:
df_op[df_op['UID']==10006]['device_code1']

In [ ]:
a=df_op[['UID','day']].groupby('UID').sum().index
b=df_tr[['UID','day']].groupby('UID').sum().index

In [ ]:
data_train=pd.DataFrame()
data_train['UID']=np.array(a | b)

In [ ]:
data_train['op_count']=0
data_train.loc[data_train['UID'].isin(df_op[['UID','day']].groupby('UID').sum().index),['op_count']]=\
df_op[['UID','day']].groupby('UID').count().values

In [ ]:
data_train['tr_count']=0
data_train.loc[data_train['UID'].isin(df_tr[['UID','day']].groupby('UID').sum().index),['tr_count']]=\
df_tr[['UID','day']].groupby('UID').count().values

In [ ]:
op_days=pd.DataFrame()
op_days['UID']=df_op['UID']
op_days=pd.concat([op_days,pd.get_dummies(df_op['day'],prefix='op_day')],axis=1)
op_days_sum=op_days.groupby(by='UID').sum()

In [ ]:
tr_days=pd.DataFrame()
tr_days['UID']=df_tr['UID']
tr_days=pd.concat([tr_days,pd.get_dummies(df_tr['day'],prefix='tr_day')],axis=1)
tr_days_sum=tr_days.groupby(by='UID').sum()

In [ ]:
data_train1=data_train

In [ ]:
data_train1=pd.merge(data_train1,op_days_sum,how='left',on='UID')

In [ ]:
data_train1.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
def op_object(train):
    for i in ['mode','os','version','']:
        train[i]=train[i].factorize()[0]
#         test[i]=test[i].factorize()[0]
        train[i]=le.fit_transform(train[i])
#         test[i]=le.fit_transform(test[i]) 
    return train


In [ ]:
op_col2=[i for i in df_op.columns.values ]
op_col2.remove('time')
op_col2.remove('day')
op_col2.remove('UID')
op_col2

In [ ]:
def fun_ob2le(train,names):
    for i in names:
        train[i]=pd.factorize(train[i])[0]
        train[i]=le.fit_transform(train[i])
    return train

In [ ]:
df_op1=df_op
df_op1=fun_ob2le(df_op1,op_col2)
df_op1.head()

In [ ]:
df_op2=pd.DataFrame()
df_op2=df_op['UID']
for i in ['success','os','version','mode']:
    df_op2=pd.concat([df_op2,pd.get_dummies(df_op1[i],prefix=i)],axis=1)   
df_op2.head()    
    

In [ ]:
df_op3=df_op2.groupby('UID',as_index=False).sum()

In [ ]:
df_op3.head()

In [ ]:
df_op1.groupby('UID').agg('nunique')

In [ ]:
df_op1[['UID','device_code1']].groupby('UID').agg('max')['device_code1'].apply(lambda x:0 if x==0 else 1)

In [ ]:
#del df_ios

In [ ]:
df_ios=df_op1[['UID','device_code1','device_code3']].groupby('UID',as_index=False).agg('max')

In [ ]:
df_ios['device_code1']=df_ios['device_code1'].apply(lambda x: 1 if x>0 else 0)
df_ios['device_code3']=df_ios['device_code3'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
df_ios.head()

In [ ]:
df_ios.rename(columns={'device_code1':'is_android','device_code3':'is_ios'},inplace=True)

In [ ]:
data_train1.shape

In [ ]:
data_train1.head()

In [ ]:
data_train2=pd.merge(data_train1,df_ios,how='left',on='UID')

In [ ]:
op1_col=df_op.columns.values.tolist()
op1_col1=[op1_col[0]]
op1_col1.extend(op1_col[7:9])
op1_col1.extend(op1_col[12:])
op1_col1


In [ ]:
df_op1.head()

In [ ]:
df_op1[df_op1['UID']==10000].head()

In [ ]:
df_op2

In [ ]:
op1_col1

In [ ]:
df_op.groupby('UID').agg('nunique')

In [ ]:
df_op1[op1_col1].groupby(by=['UID']).agg('nunique').iloc[:,1:]

In [ ]:
df_op1[op1_col1].groupby(by=['UID']).agg('nunique').iloc[:,1:]

In [ ]:
df_op4=df_op1[op1_col1].groupby(by=['UID']).agg('nunique').iloc[:,1:]

In [ ]:
df_op4.head()

In [ ]:
#########  data_train2,merge df_op3,df_op4,

In [ ]:
data_train2.merge(df_op3,how='left',on='UID').head()

In [ ]:
data_train2.merge(df_op4,how='left',on='UID').head()

In [ ]:
data_train3=data_train2.merge(df_op3,how='left',on='UID',)
data_train3=data_train3.merge(df_op4,how='left',on='UID')

In [ ]:
data_train3.head()

In [ ]:
df_tr.shape

In [ ]:
df_tr.head()

In [ ]:
df_tr[df_tr['UID']==10001].head()

In [ ]:
df_tr1=df_tr
# df_tr1_col=df_tr.columns.values.tolist()
for i in df_tr.columns.values.tolist():
    if 'UID' not in i:
        df_tr1.rename(columns={i:'tr_'+i},inplace=True)

In [ ]:
df_tr1.head()

In [ ]:
# def fun_ob2le(train,names):
#     for i in names:
#         train[i]=pd.factorize(train[i])[0]
#         train[i]=le.fit_transform(train[i])
#     return train

In [ ]:
df_tr1.columns.values.tolist()

In [ ]:
ob2le_name=[]
for i in df_tr1.columns.values.tolist():
    if  i not in ['UID','tr_day','tr_time','tr_trans_amt','tr_bal']:
        ob2le_name.append(i)
ob2le_name        

In [ ]:
df_tr2=pd.DataFrame()
df_tr2['UID']=df_tr1['UID']
df_tr2=pd.concat([df_tr2,fun_ob2le(df_tr1[ob2le_name],ob2le_name)],axis=1)

In [ ]:
df_tr2.head()

In [ ]:
# df_tr3=
df_tr3=df_tr2.groupby('UID').agg('nunique').iloc[:,1:]

In [ ]:
data_train4=data_train3.merge(df_tr3,how='left',on='UID')

In [ ]:
data_train4.head()

In [ ]:
df_tr3.head()

In [ ]:
df_tr1.head()

In [ ]:
df_tr1[['UID','tr_trans_amt','tr_bal']].groupby('UID').agg(['min','max','mean','median']).columns.values

In [ ]:
df_tr1[['UID','tr_trans_amt','tr_bal']]\
.groupby('UID').agg(['min','max','mean','median']).loc[10000:10005,[('tr_trans_amt', 'mean')]]

In [ ]:
df_tr1[['UID','tr_trans_amt','tr_bal']].groupby('UID').agg(['min','max','mean','median']).iloc[1:5,[0,1]]

In [ ]:
df_tr1.shape

In [ ]:
df_tr1[['UID','tr_trans_amt','tr_bal']].groupby('UID').agg(['min','max','mean','median']).head()

In [ ]:
df_tr3.head()

In [ ]:
data_train4.head()

In [ ]:
df_tr3_price=df_tr1[['UID','tr_trans_amt','tr_bal']].groupby('UID').agg(['min','max','mean','median'])

In [ ]:
data_train5=data_train4.merge(df_tr3_price,how='left',on='UID')

In [ ]:
data_train5.head()

In [ ]:
data_train5.columns.values

In [ ]:
for i in [('tr_trans_amt', 'min'), ('tr_trans_amt', 'max'),('tr_trans_amt', 'mean'),('tr_trans_amt', 'median'),('tr_bal', 'min'), ('tr_bal', 'max'), ('tr_bal', 'mean'),('tr_bal', 'median')]:
    data_train5.rename(columns={i:i[0]+'_'+i[1]},inplace=True)

In [ ]:
data_train5.head()

In [ ]:
data_train5[data_train5['UID']==10000][[i for i in data_train5.columns.values if 'device2' in i]]

In [8]:
df_tr2=df_tr.copy()

In [9]:
df_tr2.head()

,UID,channel,day,time,trans_amt,amt_src1,merchant,code1,code2,trans_type1,acc_id1,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,bal,amt_src2,acc_id2,acc_id3,geo_code,trans_type2,market_code,market_type,ip1_sub
0,19092,102,30,11:48:47,1459,acdbdb842ac20f1e,abc6bc660561e0eb,NaN,NaN,26bcf43a19df14c8,544dfdeedad0ae67,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,1007,9fefed0a981dcb7a,NaN,NaN,wsbq,102.0,2ecf94369847c748,1.0,45f4ca1f4e9b9cb7
1,13465,140,23,16:58:03,725,4d7831c6f695ab19,7a87b1aa20972d48,NaN,NaN,c2f2023d279665b2,NaN,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,100,NaN,NaN,NaN,wte7,105.0,NaN,NaN,2b416f1d7e89ad3b
2,13713,140,22,09:56:39,127,c5fc631370cabc0d,cfd035fc7e0a53b2,NaN,NaN,c2f2023d279665b2,NaN,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,100,NaN,NaN,NaN,ws7n,105.0,NaN,NaN,ff71a556d86c4b8f
3,22703,140,1,18:11:45,99317,a571c7fda8b7df37,c71c876b8979028e,NaN,NaN,c2f2023d279665b2,516e596377b2019c,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,100,fbf6bf3c8927414c,NaN,NaN,wqj6,NaN,NaN,NaN,cf1527a0b4173e8f
4,17816,140,22,13:04:02,3361,c5fc631370cabc0d,6628f5ca013e3582,NaN,NaN,c2f2023d279665b2,NaN,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,100,NaN,NaN,NaN,NaN,105.0,NaN,NaN,cbc62c2b1cb7deb9


In [27]:
df_tr2.update('None',filter_func=(lambda x: x.values==np.nan))

ValueError: DataFrame constructor not properly called!

In [24]:
df_tr2.values

array([[19092, 102, 30, ..., '2ecf94369847c748', 1.0, '45f4ca1f4e9b9cb7'],
       [13465, 140, 23, ..., nan, nan, '2b416f1d7e89ad3b'],
       [13713, 140, 22, ..., nan, nan, 'ff71a556d86c4b8f'],
       ...,
       [12950, 140, 15, ..., nan, nan, 'a7949b181f9b0821'],
       [12844, 102, 22, ..., nan, nan, '2ded8d34af2fb521'],
       [18910, 102, 27, ..., nan, nan, '3ac3b6d61f6389a3']], dtype=object)